In [1]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
from splinter import Browser
import time
import re

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News

In [3]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
time.sleep(10)

In [4]:
news_html = browser.html
soup = BeautifulSoup(news_html, "html.parser")

In [5]:
# Collect the latest News Title and Paragraph Text
news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_ = 'article_teaser_body').text

print(news_title)
print(news_p)

Mars Now
The agency's newest rover will use the first ground-penetrating radar instrument on the Martian surface to help search for signs of past microbial life. 


### JPL Mars Space Images - Featured Image

In [6]:
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)
time.sleep(10)

In [7]:
image_html = browser.html
image_soup = BeautifulSoup(image_html, "html.parser")

In [8]:
base_url = 'https://www.jpl.nasa.gov'
featured_image_url = image_soup.find('img', class_ = 'thumb')["src"]

In [9]:
print(base_url + featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24144-640x350.jpg


### Mars Weather

In [10]:
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(mars_weather_url)
time.sleep(10)

In [11]:
weather_html = browser.html
weather_soup = BeautifulSoup(weather_html, "html.parser")

In [12]:
tweets = weather_soup.find_all("span",text=re.compile('InSight sol'))

In [13]:
latest_tweet=tweets[0].get_text()
print(latest_tweet)

InSight sol 661 (2020-10-05) low -96.5ºC (-141.8ºF) high -13.4ºC (7.8ºF)
winds from the WNW at 6.6 m/s (14.8 mph) gusting to 21.6 m/s (48.2 mph)
pressure at 7.60 hPa


### Mars Facts

In [14]:
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)
time.sleep(10)
facts_tables = pd.read_html(facts_url)
facts_tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [15]:
facts_df = facts_tables[0]
facts_df.columns = ["Description", "Value"]
facts_df = facts_df.set_index("Description")
facts_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [16]:
html_table = facts_df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

### Mars Hemispheres

In [17]:
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)
time.sleep(10)

In [18]:
hem_html = browser.html
hemispheres_soup = BeautifulSoup(hem_html, "html.parser")

In [19]:
# results are returned as an iterable list
hemispheres = hemispheres_soup.find_all('div', class_= 'item')

In [20]:
# Loop through returned results
links = []

for hemisphere in hemispheres:
    
        # scrape the article title 
        title = hemisphere.find('h3').text

        # scrape the article image url
        img_url = hemisphere.find('a', class_= 'itemLink product-item')['href']
        browser.visit('https://astrogeology.usgs.gov' + img_url)
        time.sleep(10)
        img_url = browser.html
        imgs_url = hemisphere.find('img', class_='thumb')['src']

        # Print results only if title, price, and link are available
        if (title and imgs_url):
            hem_urls  = 'https://astrogeology.usgs.gov' + imgs_url
            links.append({'title': title, 'img_urls': hem_urls})

In [21]:
links

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_urls': 'https://astrogeology.usgs.gov/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_urls': 'https://astrogeology.usgs.gov/cache/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_urls': 'https://astrogeology.usgs.gov/cache/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_urls': 'https://astrogeology.usgs.gov/cache/images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png'}]